# Jayadeep

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def squared_error_loss(y_pred, y_true):
    return 0.5 * (y_pred - y_true) ** 2

def squared_error_loss_derivative(y_pred, y_true):
    return y_pred - y_true


#### ANN class definition

In [3]:

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Weight initialization
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        
        # Bias initialization
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.bias_output = np.zeros((1, self.output_size))
    
    def forward(self, X):
        # Input to hidden layer
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        
        # Hidden to output layer
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.output = sigmoid(self.output_input)
        
        return self.output
    
    def backward(self, X, y, output):
        # Calculate error at output
        output_error = squared_error_loss_derivative(output, y)
        output_delta = output_error * sigmoid_derivative(output)
        
        # Calculate error at hidden layer
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_output)
        
        # Update weights and biases
        self.weights_hidden_output -= self.hidden_output.T.dot(output_delta)
        self.weights_input_hidden -= X.T.dot(hidden_delta)
        
        self.bias_output -= np.sum(output_delta, axis=0, keepdims=True)
        self.bias_hidden -= np.sum(hidden_delta, axis=0, keepdims=True)

    def train_gd(self, X, y, epochs=10000):
        losses = []
        accuracies = []
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(X)
            
            # Backward pass
            self.backward(X, y, output)
            
            # Calculate loss and accuracy for tracking
            loss = np.mean(squared_error_loss(output, y))
            accuracy = np.mean((output > 0.5) == y)
            losses.append(loss)
            accuracies.append(accuracy)

            if epoch % 1000 == 0:
                print(f"Epoch {epoch} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return losses, accuracies

    def train_sgd(self, X, y, epochs=10000, batch_size=32):
        losses = []
        accuracies = []
        n_samples = X.shape[0]

        for epoch in range(epochs):
            # Shuffle dataset
            indices = np.arange(n_samples)
            np.random.shuffle(indices)

            for start in range(0, n_samples, batch_size):
                end = start + batch_size
                batch_indices = indices[start:end]
                X_batch = X[batch_indices]
                y_batch = y[batch_indices]

                # Forward pass
                output = self.forward(X_batch)
                
                # Backward pass
                self.backward(X_batch, y_batch, output)
            
            # Calculate loss and accuracy for tracking
            output_all = self.forward(X)
            loss = np.mean(squared_error_loss(output_all, y))
            accuracy = np.mean((output_all > 0.5) == y)
            losses.append(loss)
            accuracies.append(accuracy)

            if epoch % 1000 == 0:
                print(f"Epoch {epoch} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return losses, accuracies

    def predict(self, X):
        return self.forward(X)


#### Generate dataset

In [4]:

def generate_dataset(operation, n_samples=1000, noise_std=0.1):
    X = np.random.randint(0, 2, (n_samples, 2))
    y = np.zeros((n_samples, 1))
    
    if operation == 'XOR':
        y = np.logical_xor(X[:, 0], X[:, 1]).astype(int).reshape(-1, 1)
    elif operation == 'AND':
        y = np.logical_and(X[:, 0], X[:, 1]).astype(int).reshape(-1, 1)
    elif operation == 'OR':
        y = np.logical_or(X[:, 0], X[:, 1]).astype(int).reshape(-1, 1)
    
    # Add Gaussian noise
    X = X + np.random.normal(0, noise_std, X.shape)
    
    return X, y


In [5]:

# Function to run experiments
def run_experiment(operation, n_samples_list, batch_size_list, epochs=5000):
    for n_samples in n_samples_list:
        print(f"\nTraining on {operation} operation with {n_samples} samples:")
        X, y = generate_dataset(operation, n_samples)

        # Split dataset into training and testing
        split_idx = int(0.8 * len(X))
        X_train, X_test = X[:split_idx], X[split_idx:]
        y_train, y_test = y[:split_idx], y[split_idx:]

        # Initialize neural network
        nn = NeuralNetwork(input_size=2, hidden_size=2, output_size=1)

        # Train using Gradient Descent (GD)
        print("Training with Gradient Descent (GD):")
        loss_gd, accuracy_gd = nn.train_gd(X_train, y_train, epochs=epochs)

        # Evaluate on training data
        train_loss_gd = np.mean(squared_error_loss(nn.predict(X_train), y_train))
        train_accuracy_gd = np.mean((nn.predict(X_train) > 0.5) == y_train)
        print(f"GD - Training Loss: {train_loss_gd:.4f}, Training Accuracy: {train_accuracy_gd:.4f}")

        # Evaluate on testing data
        test_loss_gd = np.mean(squared_error_loss(nn.predict(X_test), y_test))
        test_accuracy_gd = np.mean((nn.predict(X_test) > 0.5) == y_test)
        print(f"GD - Test Loss: {test_loss_gd:.4f}, Test Accuracy: {test_accuracy_gd:.4f}")

        for batch_size in batch_size_list:
            print(f"Training with Stochastic Gradient Descent (SGD) with batch size {batch_size}:")
            nn = NeuralNetwork(input_size=2, hidden_size=2, output_size=1)  # Reinitialize the network
            loss_sgd, accuracy_sgd = nn.train_sgd(X_train, y_train, epochs=epochs, batch_size=batch_size)

            # Evaluate on training data
            train_loss_sgd = np.mean(squared_error_loss(nn.predict(X_train), y_train))
            train_accuracy_sgd = np.mean((nn.predict(X_train) > 0.5) == y_train)
            print(f"SGD - Training Loss: {train_loss_sgd:.4f}, Training Accuracy: {train_accuracy_sgd:.4f}")

            # Evaluate on testing data
            test_loss_sgd = np.mean(squared_error_loss(nn.predict(X_test), y_test))
            test_accuracy_sgd = np.mean((nn.predict(X_test) > 0.5) == y_test)
            print(f"SGD - Test Loss: {test_loss_sgd:.4f}, Test Accuracy: {test_accuracy_sgd:.4f}")


In [6]:

# Experiment parameters
operations = ['XOR', 'AND', 'OR']
n_samples_list = [100, 500, 1000]  # Different dataset sizes
batch_size_list = [8, 16, 32]       # Different batch sizes

# Running experiments
for operation in operations:
    run_experiment(operation, n_samples_list, batch_size_list, epochs=5000)



Training on XOR operation with 100 samples:
Training with Gradient Descent (GD):
Epoch 0 - Loss: 0.1255, Accuracy: 0.5500
Epoch 1000 - Loss: 0.0000, Accuracy: 1.0000
Epoch 2000 - Loss: 0.0000, Accuracy: 1.0000
Epoch 3000 - Loss: 0.0000, Accuracy: 1.0000
Epoch 4000 - Loss: 0.0000, Accuracy: 1.0000
GD - Training Loss: 0.0000, Training Accuracy: 1.0000
GD - Test Loss: 0.0003, Test Accuracy: 1.0000
Training with Stochastic Gradient Descent (SGD) with batch size 8:
Epoch 0 - Loss: 0.1399, Accuracy: 0.5000
Epoch 1000 - Loss: 0.0654, Accuracy: 0.8250
Epoch 2000 - Loss: 0.0649, Accuracy: 0.8250
Epoch 3000 - Loss: 0.0649, Accuracy: 0.8250
Epoch 4000 - Loss: 0.0698, Accuracy: 0.8250
SGD - Training Loss: 0.0003, Training Accuracy: 1.0000
SGD - Test Loss: 0.0005, Test Accuracy: 1.0000
Training with Stochastic Gradient Descent (SGD) with batch size 16:
Epoch 0 - Loss: 0.1270, Accuracy: 0.5000
Epoch 1000 - Loss: 0.0571, Accuracy: 0.7375
Epoch 2000 - Loss: 0.0556, Accuracy: 0.7375
Epoch 3000 - Loss: